In [1]:
# Imports and defaults
import random

"""This is used to represent an individual that is susceptible to infection."""
PRESET_STATE_HEALTHY = 0
"""This is used to represent an individual that is infected and incubating, but
is not showing symptoms, and which is not capable of transmission yet."""
PRESET_STATE_INCUBATE = 1
"""This is used to represent an individual that is currently infected and symptomatic,
and capable of infecting Healthy individuals."""
PRESET_STATE_CONTAGIOUS = 2
"""This is used to represent an individual that is currently infected and symptomatic,
but is much less likely to expose others to infection by being confined to bedrest.
Also improves the patient's chances of survival slightly."""
PRESET_STATE_BEDRIDDEN = 3
"""This is used to represent an individual that was infected, but has recovered.
This will give the individual one week of guaranteed infection immunity."""
PRESET_STATE_RECOVERED = 4
"""This is used to represent an individual that was infected and succumbed to the
disease in question."""
PRESET_STATE_DECEASED = 5

In [7]:
# Disease Parameters

"""This modifier controls the likelihood of a Contagious individual exposing
and infecting a Healthy individual. It is the chance, before any individual
resistance or connection modifiers, of infection occurring. Values are
expected to be in the range [0.05, 0.9]."""
PARAM_DISEASE_SPREAD = 0.25
"""This modifier controls the minimum number of days that the disease will
Incubate in an infected individual. Values are expected to be in the range
[1, 10]."""
PARAM_DISEASE_INCUBATION_MIN = 3
"""This modifier controls the maximum number of days that the disease can
Incubate in an infected individual. Values are expected to be in the range
[2, 14]."""
PARAM_DISEASE_INCUBATION_MAX = 10
"""This modifier controls the target number of days that the disease will be
Contagious in an infected individual. Values are expected to be in the range
[3, 21]."""
PARAM_DISEASE_CONTAGIOUS_TARGET = 10
"""This modifier controls the maximum 'progress' of the disease in an individual.
Every step of progress will slightly reduce the chance of progressing into the next,
but will also increase the chance of fatality. Values are expected to be in the
range [2, 20]. Newly-contagious individuals will start at Progress 0."""
PARAM_DISEASE_PROGRESS_MAX = 10
"""This modifier adjusts the risk of a Contagious case of the disease getting
worse for the patient. Values are expected to be in the range [0.01, 0.75]."""
PARAM_DISEASE_PROGRESS_UP = 0.3
"""This modifier adjusts the chance of a Contagious case of the disease getting
better for the patient. Values are expected to be in the range [0.01, 0.75]."""
PARAM_DISEASE_PROGRESS_DOWN = 0.15
"""This modifier controls the base risk of mortality associated with the disease.
Values are expected to be in the range [0, 0.2] for the baseline."""
PARAM_DISEASE_FATALITY_BASE = 0.002
"""This modifier controls the maximum risk of mortality associated with a very
bad case of the disease. Values are expected to be in the range [0.01, 0.9]."""
PARAM_DISEASE_FATALITY_LIMIT = 0.07

In [8]:
################################################################################
# Population Parameters

"""This modifier controls the total number of individuals in the population
being simulated. Values are expected to be in the range [10, 5000000]."""
PARAM_POPULATION_START = 4000

# The following variables pertain to "connections," which represents the 
# avenues for infection that this disease possesses.
"""This modifier controls the number of 'strong' connections that every
individual in the population is expected to have to other individuals. Values
are expected to be in the range [0, 12]."""
PARAM_POPULATION_STRONGCOUNT = 3
"""This modifier controls the number of 'weak' connections that every
individual in the population is expected to have to other individuals. Values
are expected to be in the range [1, 30]."""
PARAM_POPULATION_WEAKCOUNT = 5

"""This modifier controls the average 'health' of individuals in the population.
The individual health modifier adjusts risk of infection and fatality slightly.
Values are expected to be in the range [0.2, 0.8]."""
PARAM_POPULATION_AVGHEALTH = 0.35
"""This modifier controls the variation in the health modifiers of individuals in
the population. Values are expected to be in the range [0.05, 0.35], and the final
health modifier for any given individual will be capped to the range [0.1, 0.9]."""
PARAM_POPULATION_VARHEALTH = 0.15
"""This modifier controls the number of spontaneous exposures to attempt every day.
The likelihood of a random exposure causing infection is dependent on the proportion
of the population already infected, and on the health of the individual picked.
Values are expected to be in the range [1, 10]."""
PARAM_POPULATION_BONUS_EXPOSURES = 2
"""This modifier controls the likelihood of an infected individual being placed
into bedrest for recovery. Values are expected to be in the range [0.01, 0.4]."""
PARAM_POPULATION_BEDREST_CHANCE = 0.1
"""This modifier controls the maximum number of individuals allowed to be in bed-
rest overall. Values are expected to be in the range [1, (population / 5)]."""
PARAM_POPULATION_BEDREST_MAX = 40

In [9]:
# Simulation control parameters

"""This modifier controls the duration of the simulation, generally. Values are
expected to be in the range [10, 300]."""
PARAM_SIMULATION_DAYS = 40
"""This modifier controls the number of subdivisions (for disease progress and
infection chances) to perform during every simulation day. Values are expected
to be in the range [1, 6]."""
PARAM_SIMULATION_SUBDAYS = 3

In [17]:
"""Core Individual class. This is a single node within the undirected graph
structure that represents the simulated population. It tracks its own disease
state, strong and weak connections, and a health multiplier."""
class SimIndividual:
    def __init__(self, health: float = 0.0):
        """Construct a new Simulation Individual. If 'health' is set to 0.0, it
        will be automatically generated based on the population parameters."""
        self.state = {'main':PRESET_STATE_HEALTHY, 'subdays_active':0, 'progress':0, 
                      'update_subday_cnt':0, 'update_propensities':None}
        # 'subdays_active' tracks the amount of time the individual is in a state
        # 'progress' tracks Disease progress, if relevant
        # 'update_subday_cnt' tracks the timepoint for the next progress or state update
        # 'update_propensities' stores the propensities for the next state change, if relevant
        self.connect_strong = [None] * PARAM_POPULATION_STRONGCOUNT
        self.connect_strong_cnt = 0 # this is used to determine where we register
        self.connect_weak = [None] * PARAM_POPULATION_WEAKCOUNT
        self.connect_weak_cnt = 0
        if health == 0.0:
            self.health = min(0.9, max(0.1, random.gauss(PARAM_POPULATION_AVGHEALTH, PARAM_POPULATION_VARHEALTH)))
        else:
            self.health = health
            
    def assign_connection(self, who, strong: bool = False) -> bool:
        """Attempt to add a two-way connection between this Individual and the
        specified target Individual, of the specified strength level. Returns
        True if both Individuals had an open slot for the connection and one
        was successfully established, False otherwise."""
        if strong:
            selfallowed = self.connect_strong_cnt < PARAM_POPULATION_STRONGCOUNT
            themallowed = who.connect_strong_cnt < PARAM_POPULATION_STRONGCOUNT
            if selfallowed and themallowed:
                self.connect_strong[self.connect_strong_cnt] = who
                who.connect_strong[who.connect_strong_cnt] = self
                self.connect_strong_cnt += 1
                who.connect_strong_cnt += 1
                return True
            else:
                return False
        else:
            selfallowed = self.connect_weak_cnt < PARAM_POPULATION_WEAKCOUNT
            themallowed = who.connect_weak_cnt < PARAM_POPULATION_WEAKCOUNT
            if selfallowed and themallowed:
                self.connect_weak[self.connect_weak_cnt] = who
                who.connect_weak[who.connect_weak_cnt] = self
                self.connect_weak_cnt += 1
                who.connect_weak_cnt += 1
                return True
            else:
                return False

    def change_core_state(self, new_state: int) -> bool:
        """Attempt to change the current simulation state of this Individual to
        the specified integer, representing one of the PRESET_STATE entries.
        Returns True if the state was modified, False otherwise."""
        if self.state['main'] == new_state:
            return False
        else:
            self.state['main'] = new_state
            self.state['last_subdays'] = self.state['subdays_active']
            self.state['subdays_active'] = 0
            # perform "first change" logic for disease states
            if new_state == PRESET_STATE_INCUBATE:
                targetnum = random.randint(PARAM_SIMULATION_SUBDAYS * PARAM_DISEASE_INCUBATION_MIN,\
                                           PARAM_SIMULATION_SUBDAYS * PARAM_DISEASE_INCUBATION_MAX)
                self.state['update_subday_cnt'] = targetnum
            elif new_state == PRESET_STATE_CONTAGIOUS:
                self.state['update_propensities'] = self.disease_substate_nextgoal()
            elif new_state == PRESET_STATE_RECOVERED:
                self.state['update_subday_cnt'] = PARAM_SIMULATION_SUBDAYS * 7
            return True

    def disease_substate_nextgoal(self) -> tuple:
        """Determines the number of subdays until the next expected disease update,
        and assigns that to this Individual's state. Returns a tuple, containing
        the propensities for their condition worsening."""
        
        subday_fraction = self.state['subdays_active'] / (PARAM_DISEASE_CONTAGIOUS_TARGET * PARAM_SIMULATION_SUBDAYS)
        if self.state['progress']:
            prop_up = PARAM_DISEASE_PROGRESS_UP * (1 + 0.1 * (PARAM_DISEASE_PROGRESS_MAX - self.state['progress'])) \
                    * (1 - 0.1 * self.health) / (1 + 4 * subday_fraction)
            prop_dn = PARAM_DISEASE_PROGRESS_DOWN * (1 + subday_fraction) * (1 + 0.1 * self.health) \
                    * (0.2 + 0.5 * self.state['progress'] / PARAM_DISEASE_PROGRESS_MAX)
            
            target = round(PARAM_SIMULATION_SUBDAYS * random.expovariate(1.0 / (prop_up + prop_dn)))
            self.state['update_subday_cnt'] = target
            return (prop_up, prop_dn)
        else:
            target = round(PARAM_SIMULATION_SUBDAYS * random.expovariate(1.0 / PARAM_DISEASE_PROGRESS_UP))
            self.state['update_subday_cnt'] = target
            return (PARAM_DISEASE_PROGRESS_UP, 0.0)

    def debug_print(self):
        print(self.state, self.health, self.connect_strong_cnt, self.connect_weak_cnt)
    
    def state_subday_simulate(self):
        """Perform one sub-day of simulation for this Individual's state."""
        self.state['subdays_active'] += 1
        statenum = self.state['main']
        if statenum and self.state['subdays_active'] >= self.state['update_subday_cnt']:
            if statenum == PRESET_STATE_INCUBATE:
                # Disease progresses from Incubating to Contagious
                self.change_core_state(PRESET_STATE_CONTAGIOUS)
            elif statenum == PRESET_STATE_CONTAGIOUS:
                worsens = random.choices([True, False], self.state['update_propensities'])[0]
                if worsens:
                    self.state['progress'] += 1
                    if self.state['progress'] >= PARAM_DISEASE_PROGRESS_MAX:
                        self.state['progress'] = PARAM_DISEASE_PROGRESS_MAX
                else:
                    self.state['progress'] -= 1
                    if self.state['progress'] <= 0:
                        self.change_core_state(PRESET_STATE_RECOVERED)
                self.state['update_propensities'] = self.disease_substate_nextgoal()
                deathrisk = PARAM_DISEASE_FATALITY_BASE + ((PARAM_DISEASE_FATALITY_LIMIT - PARAM_DISEASE_FATALITY_BASE) \
                            * (self.state['progress'] / PARAM_DISEASE_PROGRESS_MAX))
                deathrisk *= (1.0 - 0.25 * self.health) / (1 + PARAM_SIMULATION_SUBDAYS) # fatality_base is in daily terms

                if random.uniform(0.0, 1.0) < deathrisk:
                    self.change_core_state(PRESET_STATE_DECEASED)
                    return
                
                # TO-DO: implement bedridden

# get some stats for starting evaluation
times_to_recovery = 0
times_recovered = 0
times_to_death = 0
times_died = 0

for iteration in range(500):    
    rob = SimIndividual(0.15)
    rob.change_core_state(PRESET_STATE_INCUBATE)
    for i in range(PARAM_SIMULATION_SUBDAYS * PARAM_SIMULATION_DAYS):
        rob.state_subday_simulate()
        if rob.state['main'] == 5:
            times_to_death += rob.state['last_subdays']
            times_died += 1
            break
        elif rob.state['main'] == 4:
            times_to_recovery += rob.state['last_subdays']
            times_recovered += 1
            break

print("For 500 simulations,", times_recovered, "recovered and", times_died, "died.")
print("The average recovery time was", times_to_recovery / times_recovered, "subdays (", times_to_recovery \
    / (times_recovered * PARAM_SIMULATION_SUBDAYS), "days)")
print("The average death time was", times_to_death / times_died, "subdays (", times_to_death \
    / (times_died * PARAM_SIMULATION_SUBDAYS), "days)")
print("Fatality parameters were", PARAM_DISEASE_FATALITY_BASE, "base and", PARAM_DISEASE_FATALITY_LIMIT, "max")
print("Target disease duration was", PARAM_DISEASE_CONTAGIOUS_TARGET, "days")

For 500 simulations, 366 recovered and 134 died.
The average recovery time was 40.53005464480874 subdays ( 13.510018214936247 days)
The average death time was 23.62686567164179 subdays ( 7.875621890547263 days)
Fatality parameters were 0.002 base and 0.07 max
Target disease duration was 10 days


In [6]:
################################################################################
# A Class for the simulator itself, which generates individuals and updates 
# their states one-by-one
class Simulator:
    def __init__(self, population_size):
        """Creates a simulator object, which will generate individuals, tell 
        them to update their states, and collect data on the states of 
        individuals."""
        self.individuals = [None] * population_size
        self.individual_counts_in_each_state = {
            PRESET_STATE_HEALTHY: 0,
            PRESET_STATE_INCUBATE: 0,
            PRESET_STATE_CONTAGIOUS: 0,
            PRESET_STATE_BEDRIDDEN: 0,
            PRESET_STATE_RECOVERED: 0,
            PRESET_STATE_DECEASED: 0
        }
        self.simulation_length = PARAM_SIMULATION_DAYS * PARAM_SIMULATION_SUBDAYS

        # Store a population's worth of individuals into the individuals list
        for i in range(population_size):
            self.individuals[i] = SimIndividual()
